# Getting POI Data

In [1]:
#Importing the important libraries
import pandas as pd
import geopandas as gp
# OSMNX for getting OpenStreetMap Data
import osmnx as ox
# Folium for plotting maps
import folium

In [2]:
# Import POIs with an amenity tag
# https://wiki.openstreetmap.org/wiki/Key:amenity
tags = {'amenity': True}
poi = ox.geometries_from_place(query = {'city': 'Chicago'}, tags=tags)

# As we will see later not all public transport stops are marked with an amenity tag
# Therefore we will load a second datasetfor public transport related POIs
# https://wiki.openstreetmap.org/wiki/Key:public_transport
tags = {'public_transport': True}
transport = ox.geometries_from_place(query = {'city': 'Chicago'}, tags=tags)

C:\Users\phili\AppData\Local\Temp\ipykernel_18044\3943264863.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  poi = ox.geometries_from_place(query = {'city': 'Chicago'}, tags=tags)
C:\Users\phili\AppData\Local\Temp\ipykernel_18044\3943264863.py:10: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  transport = ox.geometries_from_place(query = {'city': 'Chicago'}, tags=tags)


### Amenity Data

In [3]:
# As we loaded the public transport data as a second dataframe we now exclude the double entries
poi = poi[poi["public_transport"].isna()]

In [4]:
# Reduce dataframe to important columns
poiDF = poi[["amenity", "geometry", "name"]].reset_index().set_index("osmid").drop("element_type", axis = 1).copy()
poiDF.tail(3)

,amenity,geometry,name
osmid,,,
15910908,shelter,"POLYGON ((-87.62616 41.87921, -87.62615 41.878...",NaN
15925258,theatre,"POLYGON ((-87.62925 41.88838, -87.62908 41.888...",House of Blues
15979377,university,"MULTIPOLYGON (((-87.62757 41.87828, -87.62758 ...",DePaul University (the Loop Campus)


In [5]:
print(len(poiDF['amenity'].unique()))

121


In [5]:
# Have a look at the different types of POI and their occurrence
print(poiDF.groupby("amenity").count()["geometry"].sort_values(ascending=False).to_string())

amenity
parking                    6659
restaurant                 2212
place_of_worship           1873
fast_food                  1274
school                     1214
bicycle_parking             878
bar                         854
bicycle_rental              817
bench                       799
cafe                        760
bank                        455
fuel                        418
parking_entrance            330
post_box                    231
shelter                     228
toilets                     224
fountain                    206
social_facility             198
pharmacy                    181
drinking_water              179
loading_dock                178
clinic                      145
waste_basket                127
library                     108
fire_station                103
ice_cream                   102
pub                         101
post_office                 100
vending_machine              98
atm                          95
studio                       92


In [6]:
# For later analyses it could be interesting to group some tags and make a simpler categorization
# We don't group some items like for example transport related tags as their differences could be of importance
# Parking & Transport related POIs
car_parking = ["parking", "parking_entrance", 'parking_space']
bicycle_parking = ["bicycle_parking"]
bicycle_rental = ["bicycle_rental"]
bicycle_repair_station  =  ['bicycle_repair_station']
taxi = ['taxi']
car_sharing = ['car_sharing']
car_rental = ['car_rental']
water_transport = ["water_transport",'loading_dock', 'boat_rental','boat_storage']
fuel = ["fuel"]
# Other maybe important POIs
gastronomy = ["gastronomy", 'fast_food', 'cafe', 'restaurant', 'pub', 'bar', 'ice_cream', 'food_court', 'fast_food;ice_cream', 'biergarten']
place_of_worship = ['place_of_worship']
finance = ["finance", 'bank', 'atm', 'bureau_de_change', 'money_transfer', 'payment_terminal', 'payment_centre']
sports = ["sports", 'pilates', 'training']
health = ["health", 'pharmacy', 'hospital', 'dentist', 'clinic', 'nursing_home', 'doctors']
education = ["education", "music_school", 'school', 'university', 'college', 'language_school', 'library', 'prep_school', 'flight_school','research_institute']
nightlife = ["nightlife", 'stripclub', 'nightclub']
culture = ["culture", 'theatre', 'cinema', 'events_venue', 'planetarium', 'music_venue', 'arts_centre', 'exhibition_centre']
children = ["children", 'childcare', "kindergarten"]
police = ["police"]

category_list = [
car_parking,
bicycle_parking,
bicycle_rental,
bicycle_repair_station,
taxi,
car_sharing,
car_rental,
water_transport,
fuel,
gastronomy,
place_of_worship,
finance,
sports,
health,
education,
nightlife,
culture,
children,
police]

In [7]:
# Method to apply our categorization to a row of the dataframe
def simpleCategory(row):
    for x in category_list:
        if row["amenity"] in x:
            return x[0]
    return "other"

In [8]:
# Create a column with our categorization
poiDF["category"] = poiDF.apply(simpleCategory, axis=1)

In [9]:
# Have a look at the resulting occurences of categories
print(poiDF.groupby("category").count()["geometry"].sort_values(ascending=False).to_string())

category
parking                   7037
gastronomy                5324
other                     3055
place_of_worship          1873
education                 1428
bicycle_parking            878
bicycle_rental             817
finance                    597
health                     494
fuel                       418
water_transport            185
culture                    180
children                    58
police                      38
nightlife                   24
car_rental                  23
bicycle_repair_station      19
taxi                        16
car_sharing                  2
sports                       2


In [10]:
# Method that returns the Latitude and longitude
def getLatAndLong(row):
    if (row.geometry.geom_type == "Point"):
        return [row.geometry.y, row.geometry.x]
    # For all POIs that are not only points, but polygons or multigons we return the center coordinates
    else:
        return [row.geometry.centroid.y, row.geometry.centroid.x]

In [11]:
# Add columns with latitude and longitude
poiDF[["latitude", "longitude"]] = poiDF.apply(getLatAndLong, axis=1, result_type='expand')

In [12]:
poiDF.tail(3)

,amenity,geometry,name,category,latitude,longitude
osmid,,,,,,
15910908,shelter,"POLYGON ((-87.62616 41.87921, -87.62615 41.878...",NaN,other,41.879540,-87.626136
15925258,theatre,"POLYGON ((-87.62925 41.88838, -87.62908 41.888...",House of Blues,culture,41.888259,-87.629116
15979377,university,"MULTIPOLYGON (((-87.62757 41.87828, -87.62758 ...",DePaul University (the Loop Campus),education,41.877826,-87.626584


### Public Transport Data

In [13]:
# Reduce dataframe to important columns
transportDF = transport[["public_transport", "wheelchair", "geometry", "name", "amenity"]].reset_index().set_index("osmid").drop("element_type", axis = 1)

In [15]:
transportDF.head(3)

,public_transport,wheelchair,geometry,name,amenity
osmid,,,,,
20217109,station,NaN,POINT (-87.62252 41.88915),Michigan Avenue Bridge Dock,ferry_terminal
20217237,stop_position,NaN,POINT (-87.63177 41.88782),River North Water Taxi Terminal,ferry_terminal
20217442,stop_position,NaN,POINT (-87.63774 41.87906),Willis Tower/Union Station Water Taxi Terminal,ferry_terminal


In [14]:
# Look at different type of "public_transport" tags
print(transportDF.groupby("public_transport").count()["geometry"].sort_values(ascending=False).to_string())

public_transport
platform          10656
stop_position      5524
station             264
no                   19
service_center        3


In [15]:
# https://wiki.openstreetmap.org/wiki/Key:public_transport
# Only keep relevant entries
transportDF = transportDF[transportDF["public_transport"].isin(["platform", "stop_position", "station"])].copy()

In [16]:
# Add columns with latitude and longitude
transportDF[["latitude", "longitude"]] = transportDF.apply(getLatAndLong, axis=1, result_type='expand')

In [19]:
transportDF.head(3)

,public_transport,wheelchair,geometry,name,amenity,latitude,longitude
osmid,,,,,,,
20217109,station,NaN,POINT (-87.62252 41.88915),Michigan Avenue Bridge Dock,ferry_terminal,41.889146,-87.622517
20217237,stop_position,NaN,POINT (-87.63177 41.88782),River North Water Taxi Terminal,ferry_terminal,41.887824,-87.631768
20217442,stop_position,NaN,POINT (-87.63774 41.87906),Willis Tower/Union Station Water Taxi Terminal,ferry_terminal,41.879062,-87.637740


### Merging the two dataframes

In [17]:
transportDF["category"] = "public_transport"
transportDF.head(3)

,public_transport,wheelchair,geometry,name,amenity,latitude,longitude,category
osmid,,,,,,,,
20217109,station,NaN,POINT (-87.62252 41.88915),Michigan Avenue Bridge Dock,ferry_terminal,41.889146,-87.622517,public_transport
20217237,stop_position,NaN,POINT (-87.63177 41.88782),River North Water Taxi Terminal,ferry_terminal,41.887824,-87.631768,public_transport
20217442,stop_position,NaN,POINT (-87.63774 41.87906),Willis Tower/Union Station Water Taxi Terminal,ferry_terminal,41.879062,-87.637740,public_transport


In [18]:
df = pd.concat([poiDF, transportDF], axis = 0, join = "outer")
df

,amenity,geometry,name,category,latitude,longitude,public_transport,wheelchair
osmid,,,,,,,,
258490031,parking,POINT (-87.64385 41.95123),NaN,parking,41.951227,-87.643847,NaN,NaN
269449042,parking_entrance,POINT (-87.61506 41.85869),NaN,parking,41.858689,-87.615058,NaN,NaN
269450074,parking_entrance,POINT (-87.58421 41.79174),NaN,parking,41.791742,-87.584208,NaN,NaN
269688947,post_box,POINT (-87.77220 42.00407),NaN,other,42.004073,-87.772198,NaN,NaN
270863340,parking_entrance,POINT (-87.61564 41.86421),NaN,parking,41.864208,-87.615638,NaN,NaN
...,...,...,...,...,...,...,...,...
13621412,NaN,"POLYGON ((-87.66859 41.91735, -87.66895 41.917...",NaN,public_transport,41.917203,-87.668448,platform,NaN
13621413,NaN,"POLYGON ((-87.66789 41.91673, -87.66793 41.916...",NaN,public_transport,41.917205,-87.668177,platform,NaN
13621414,NaN,"POLYGON ((-87.66801 41.91728, -87.66800 41.917...",NaN,public_transport,41.917277,-87.668066,platform,NaN


In [19]:
print(df.groupby("category").count()["geometry"].sort_values(ascending=False).to_string())

category
public_transport          16444
parking                    7037
gastronomy                 5324
other                      3055
place_of_worship           1873
education                  1428
bicycle_parking             878
bicycle_rental              817
finance                     597
health                      494
fuel                        418
water_transport             185
culture                     180
children                     58
police                       38
nightlife                    24
car_rental                   23
bicycle_repair_station       19
taxi                         16
car_sharing                   2
sports                        2


### Saving dataframe

In [20]:
df.to_pickle("data/poi/poi.pickle")

## Hexagons

In [21]:
# Importing Libarys For Spatial Discretization
import h3
from shapely import Polygon 

In [22]:
df = pd.read_pickle("data/poi/poi.pickle")

In [26]:
df

,amenity,geometry,name,category,latitude,longitude,public_transport,wheelchair
osmid,,,,,,,,
258490031,parking,POINT (-87.64385 41.95123),NaN,parking,41.951227,-87.643847,NaN,NaN
269449042,parking_entrance,POINT (-87.61506 41.85869),NaN,parking,41.858689,-87.615058,NaN,NaN
269450074,parking_entrance,POINT (-87.58421 41.79174),NaN,parking,41.791742,-87.584208,NaN,NaN
269688947,post_box,POINT (-87.77220 42.00407),NaN,other,42.004073,-87.772198,NaN,NaN
270863340,parking_entrance,POINT (-87.61564 41.86421),NaN,parking,41.864208,-87.615638,NaN,NaN
...,...,...,...,...,...,...,...,...
13621412,NaN,"POLYGON ((-87.66859 41.91735, -87.66895 41.917...",NaN,public_transport,41.917203,-87.668448,platform,NaN
13621413,NaN,"POLYGON ((-87.66793 41.91671, -87.66797 41.916...",NaN,public_transport,41.917215,-87.668182,platform,NaN
13621414,NaN,"POLYGON ((-87.66801 41.91728, -87.66800 41.917...",NaN,public_transport,41.917277,-87.668066,platform,NaN


In [23]:
def convert_to_hex_id (row, res):
    hex_id = h3.latlng_to_cell(row["latitude"], row["longitude"], res)
    return hex_id if hex_id != "0" else None

def convert_hex_to_polygon(hex):
        polygon = None
        if hex:
               polygon = Polygon(h3.cell_to_boundary(hex, geo_json=True))         
        return polygon

In [24]:
# Creating Hex_ID Columns
df['hex_7']  = df.apply(convert_to_hex_id, res=7, axis=1)
df['hex_8']  = df.apply(convert_to_hex_id, res=8, axis=1)
df['hex_9']  = df.apply(convert_to_hex_id, res=9, axis=1)

In [25]:
# Creating Polygon Columns
df['pol_7']  = df['hex_7'].apply(convert_hex_to_polygon)
df['pol_8']  = df['hex_8'].apply(convert_hex_to_polygon) 
df['pol_9']  = df['hex_9'].apply(convert_hex_to_polygon) 

In [30]:
df

,amenity,geometry,name,category,latitude,longitude,public_transport,wheelchair,hex_7,hex_8,hex_9,pol_7,pol_8,pol_9
osmid,,,,,,,,,,,,,,
258490031,parking,POINT (-87.64385 41.95123),NaN,parking,41.951227,-87.643847,NaN,NaN,872664c14ffffff,882664c145fffff,892664c144fffff,"POLYGON ((-87.63414405293 41.96445315471353, -...",POLYGON ((-87.64084295487153 41.95798939959579...,POLYGON ((-87.64331225830723 41.95215193946119...
269449042,parking_entrance,POINT (-87.61506 41.85869),NaN,parking,41.858689,-87.615058,NaN,NaN,872664c1bffffff,882664c1b1fffff,892664c1b03ffff,"POLYGON ((-87.61820944356228 41.8710984903598,...",POLYGON ((-87.61347038938916 41.86360034164272...,POLYGON ((-87.61470441038506 41.86068519734808...
269450074,parking_entrance,POINT (-87.58421 41.79174),NaN,parking,41.791742,-87.584208,NaN,NaN,872664cc5ffffff,882664cc59fffff,892664cc583ffff,POLYGON ((-87.5937067068291 41.798228672497444...,POLYGON ((-87.58229602405846 41.79718226760902...,POLYGON ((-87.58353033145704 41.79426977332311...
269688947,post_box,POINT (-87.77220 42.00407),NaN,other,42.004073,-87.772198,NaN,NaN,872664d9effffff,882664d9e1fffff,892664d9e0fffff,"POLYGON ((-87.7773181551225 42.01982017726515,...",POLYGON ((-87.77253980099742 42.01231761635508...,POLYGON ((-87.77499453418736 42.00647434498914...
270863340,parking_entrance,POINT (-87.61564 41.86421),NaN,parking,41.864208,-87.615638,NaN,NaN,872664c1bffffff,882664c1bdfffff,892664c1bc7ffff,"POLYGON ((-87.61820944356228 41.8710984903598,...","POLYGON ((-87.61820944356228 41.8710984903598,...","POLYGON ((-87.6164567332772 41.86589170573153,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13621412,NaN,"POLYGON ((-87.66859 41.91735, -87.66895 41.917...",NaN,public_transport,41.917203,-87.668448,platform,NaN,872664c12ffffff,882664cac9fffff,892664cac93ffff,"POLYGON ((-87.6723729663648 41.93963771855481,...","POLYGON ((-87.6695545771894 41.9181739690806, ...","POLYGON ((-87.6695545771894 41.9181739690806, ..."
13621413,NaN,"POLYGON ((-87.66793 41.91671, -87.66797 41.916...",NaN,public_transport,41.917215,-87.668182,platform,NaN,872664c12ffffff,882664cac9fffff,892664cac93ffff,"POLYGON ((-87.6723729663648 41.93963771855481,...","POLYGON ((-87.6695545771894 41.9181739690806, ...","POLYGON ((-87.6695545771894 41.9181739690806, ..."
13621414,NaN,"POLYGON ((-87.66801 41.91728, -87.66800 41.917...",NaN,public_transport,41.917277,-87.668066,platform,NaN,872664c12ffffff,882664cac9fffff,892664cac93ffff,"POLYGON ((-87.6723729663648 41.93963771855481,...","POLYGON ((-87.6695545771894 41.9181739690806, ...","POLYGON ((-87.6695545771894 41.9181739690806, ..."


This is the final POI-DataFrame. Lets look at the columns.


| column | explanation |
|---|---|
| osmid | OSM ID of the POI |
| amenity | "amenity=* is the top-level tag describing useful and important facilities for visitors and residents, such as toilets, telephones, banks, pharmacies, prisons and schools" - [OSM Wiki](https://wiki.openstreetmap.org/wiki/Main_Page)|
| geometry | Shape and position of the POI. |
| name | Some POIs are named, for example "Hilbert Pub" |
| category | The selfmade categories for easy later analysis |
| latitude | Latitude |
| longitude | Longitude |
| public_transport | "The public_transport key denotes stop positions and platforms of public transport" - [OSM Wiki](https://wiki.openstreetmap.org/wiki/Main_Page)|
| wheelchair | "This tag may be used to mark places or ways that are suitable to be used with a wheelchair and a person with a disability who uses another mobility device (like a walker)" - [OSM Wiki](https://wiki.openstreetmap.org/wiki/Main_Page)|
| hex_7 | HexagonID for resolution 7 |
| hex_8 | HexagonID for resolution 8 |
| hex_9 | HexagonID for resolution 9 |
| pol_7 | Geometry of Heaxagon for resolution 7 |
| pol_8 | Geometry of Heaxagon for resolution 8 |
| pol_9 | Geometry of Heaxagon for resolution 9 |

In [26]:
df.to_pickle("data/poi/poi_prepared.pickle")